In [186]:
import pdfquery
import datetime
import pandas as pd

In [514]:
def extract_text(l):
    return "".join([k for k in l if not k.isdigit() and not k in [".", ",", "+"]]).rstrip()
def extract_numbers(l, el=None):
#     lis = "".join([k for k in l if k.isdigit() or k in [" "]]).lstrip().rstrip().split(" ")
#     print(lis)
#     print([m for m in lis if len(m) > 0])
    result = [int(m) for m in "".join([k for k in l if k.isdigit() or k in [" "]]).lstrip().rstrip().split(" ") if len(m) > 0]
    if len(result) < 4:
        result += [0]*(4-len(result))
    if el is None:
        return result
    else:
        return result[el]
def count_numbers(l):
    c = 0
    try:
        c += sum([1 for k in l if k.isdigit()])
        return c
    except:
        return c
def extract_number(l):
    result = "".join([k for k in l.values[0] if k.isdigit()]).lstrip().rstrip()
    if len(result) > 0:
        return int(result)
    else:
        return 0
# df.loc[:, ['confirmed']].apply(extract_number, axis=1)

In [515]:
def load_pdf(date, path="data/RKI-reports/{0}-{1}.pdf", lang="en"):
    from tabula import read_pdf
    path = path.format(date, lang)
    print(path)
    df = read_pdf(path, pages=[2])
    df = df[0].iloc[4:]
    try:
        df.columns = ['data']
        df['c'] = df.loc[:, 'data'].apply(count_numbers)
        df = df[df.c != 0]
    
        df['land'] = df.loc[:, 'data'].apply(extract_text)
        df['confirmed'] = df.loc[:, 'data'].apply(extract_numbers, args=[0])
        df['daily'] = df.loc[:, 'data'].apply(extract_numbers, args=[1])
        df['per_mil'] = df.loc[:, 'data'].apply(extract_numbers, args=[2])
        df['dead'] = df.loc[:, 'data'].apply(extract_numbers, args=[3])
    except:
        if len(df.columns) == 4:
            df.columns = ['land', 'confirmed', 'daily', 'dead']
            df['per_mil'] = 0
            df = df.loc[:, ['land', 'confirmed', 'daily', 'per_mil', 'dead']]
        elif len(df.columns) == 5:
            df.columns = ['land', 'confirmed', 'daily', 'per_mil', 'dead']
        else:
            print(df)
            raise Exception
        df['c'] = df.loc[:, 'confirmed'].apply(count_numbers)
        df = df[df.c != 0]
        df.loc[:,['confirmed']] = df.loc[:,['confirmed']].apply(extract_number, axis=1)
        df.loc[:,['daily']] = df.loc[:,['daily']].apply(extract_number, axis=1)
        df.loc[:,['dead']] = df.loc[:,['dead']].apply(extract_number, axis=1)
    df = df[~df.land.str.contains('cases')]
    df = df[df.land != 'Total']
    try:
        df['date'] = datetime.datetime.strptime(date, "%Y-%m-%d")
    except:
        df['date'] = date
    return df

# Test Load

In [551]:
load_pdf('2020-04-18', lang="en").head()
# df = load_pdf('2020-03-18', lang="en")

data/RKI-reports/2020-04-18-en.pdf


,data,c,land,confirmed,daily,per_mil,dead,date
4,"Baden-Wuerttemberg 27,258 715 246 919",14,Baden-Wuerttemberg,27258,715,246,919,2020-04-18
5,"Bavaria 36,881 854 282 1,198",15,Bavaria,36881,854,282,1198,2020-04-18
6,"Berlin 5,066 121 135 91",12,Berlin,5066,121,135,91,2020-04-18
7,"Brandenburg 2,161 41 86 62",10,Brandenburg,2161,41,86,62,2020-04-18
8,Bremen 567 11 83 24,9,Bremen,567,11,83,24,2020-04-18


# Batch Load

In [519]:
df = pd.DataFrame(columns=['data', 'land', 'confirmed', 'daily', 'per_mil', 'dead'])
list_df = []
start='2020-03-18'
end='2020-04-17'
for d in pd.date_range(start, end).date:
    list_df.append(load_pdf(d, path="data/RKI-reports/{0}-{1}.pdf", lang="en"))

data/RKI-reports/2020-03-18-en.pdf
data/RKI-reports/2020-03-19-en.pdf
data/RKI-reports/2020-03-20-en.pdf
data/RKI-reports/2020-03-21-en.pdf
data/RKI-reports/2020-03-22-en.pdf
data/RKI-reports/2020-03-23-en.pdf
data/RKI-reports/2020-03-24-en.pdf
data/RKI-reports/2020-03-25-en.pdf
data/RKI-reports/2020-03-26-en.pdf
data/RKI-reports/2020-03-27-en.pdf
data/RKI-reports/2020-03-28-en.pdf
data/RKI-reports/2020-03-29-en.pdf
data/RKI-reports/2020-03-30-en.pdf
data/RKI-reports/2020-03-31-en.pdf
data/RKI-reports/2020-04-01-en.pdf
data/RKI-reports/2020-04-02-en.pdf
data/RKI-reports/2020-04-03-en.pdf
data/RKI-reports/2020-04-04-en.pdf
data/RKI-reports/2020-04-05-en.pdf
data/RKI-reports/2020-04-06-en.pdf
data/RKI-reports/2020-04-07-en.pdf
data/RKI-reports/2020-04-08-en.pdf
data/RKI-reports/2020-04-09-en.pdf
data/RKI-reports/2020-04-10-en.pdf
data/RKI-reports/2020-04-11-en.pdf
data/RKI-reports/2020-04-12-en.pdf
data/RKI-reports/2020-04-13-en.pdf
data/RKI-reports/2020-04-14-en.pdf
data/RKI-reports/202

## Batch Save

In [544]:
result_df = pd.concat(list_df).drop(['daily', 'c', 'per_mil'], axis=1)

In [545]:
result_df.to_csv('data/RKI-reports.csv', index=False)

-----

-----

# Updating Reports

In [651]:
df_new = load_pdf('2020-04-25', lang="en")
df_new.drop(['c', 'daily', 'per_mil'], axis=1, inplace=True)

data/RKI-reports/2020-04-25-en.pdf


In [652]:
result_all = pd.read_csv('data/RKI-reports.csv')
result_all['date'] = result_all['date'].astype('datetime64[ns]')

In [653]:
result_concat = pd.concat([result_all, df_new]).sort_values('date', ascending=False)
result_concat.head()

,land,confirmed,dead,date,data
19,Thuringia,2058,71,2020-04-25,Thuringia 2.058 35 96 71
11,Mecklenburg-Western Pomerania,667,16,2020-04-25,Mecklenburg-Western Pomerania 667 6 41 16
5,Bavaria,40547,1566,2020-04-25,Bavaria 40.547 608 310 1.566
6,Berlin,5525,113,2020-04-25,Berlin 5.525 66 147 113
7,Brandenburg,2627,97,2020-04-25,Brandenburg 2.627 91 105 97


In [654]:
result_concat = result_concat[result_concat.land != 'Federal State Total Number Number of Cases/ Number of']

In [655]:
result_concat.to_csv('data/RKI-reports.csv', index=False)

---

In [558]:
result_concat.loc[:, ['land', 'date', 'confirmed']].groupby(['land', 'date']).count().sort_values(['confirmed'], ascending=False).head()

confirmed
land                 date                 
Baden-Wuerttemberg   2020-03-26          1
Rhineland-Palatinate 2020-04-04          1
                     2020-04-17          1
                     2020-04-16          1
                     2020-04-15          1

In [625]:
result_concat.loc[:, ['land', 'confirmed']].groupby(['land']).count().sort_values(['confirmed'], ascending=False).head(50)

,confirmed
land,
Saxony,34
Bavaria,33
Berlin,33
Brandenburg,33
Bremen,33
Hamburg,33
Hesse,33
Lower Saxony,33
North Rhine-Westphalia,33


In [614]:
fixed_cresult = result_concat.copy(deep=True)
fixed_cresult.loc[fixed_cresult['land'] == 'Mecklenburg-Western', 'land'] = 'Mecklenburg-Western Pomerania'
fixed_cresult = fixed_cresult[fixed_cresult['land'] != 'Mecklenburg-Western']

In [615]:
fixed_cresult.loc[fixed_cresult['land'] == 'Saxony-Anhalt*', 'land'] = 'Saxony'
fixed_cresult = fixed_cresult[fixed_cresult['land'] != 'Saxony-Anhalt*']

In [616]:
fixed_cresult.loc[fixed_cresult['land'] == 'Baden-Württemberg', 'land'] = 'Baden-Wuerttemberg'
fixed_cresult = fixed_cresult[fixed_cresult['land'] != 'Baden-Württemberg']

In [617]:
fixed_cresult.loc[:, ['land', 'confirmed']].groupby(['land']).count().sort_values(['confirmed'], ascending=False).head(50)


,confirmed
land,
Saxony,33
Bavaria,32
Berlin,32
Brandenburg,32
Bremen,32
Hamburg,32
Hesse,32
Lower Saxony,32
North Rhine-Westphalia,32


---

In [619]:
fixed_cresult.to_csv('data/RKI-reports.csv', index=False)

----